In [28]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [29]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [30]:
INPUT_DIR = "output/20240805-112918/artifacts/"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [31]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 2524


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,entity_type,top_level_node_id,x,y
0,0,BÁN CHỨNG KHOÁN LÔ LẺ,DỊCH VỤ GIAO DỊCH,Bán chứng khoán lô lẻ là dịch vụ giao dịch chứ...,72c428d19c253035db6c6e314742ef8c,12,1,0,b45241d70f0e43fca764df95b2b81f77,1,None,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,MBS,CÔNG TY CHỨNG KHOÁN,"MBS, also known as Công ty CP Chứng khoán MBS,...","008d65e4640a5208f1016c02bde466fa,0155a5af5c198...",12,122,1,4119fd06010c494caa07f439b333f4c5,122,None,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,CỔ PHIẾU LÔ LẺ,CỔ PHIẾU,Cổ phiếu lô lẻ là cổ phiếu có khối lượng từ 1 ...,72c428d19c253035db6c6e314742ef8c,1,1,2,d3835bf3dda84ead99deadbeac5d0d7d,1,None,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,NHÀ ĐẦU TƯ,NGƯỜI,"""NHÀ ĐẦU TƯ"" là những cá nhân hoặc tổ chức tha...","078f14bae4d779e6ad5fad23e7f2ea43,07ee7a0c91297...",1,57,3,077d2820ae1845bcbb1803379a3d1eae,57,None,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,SỞ GIAO DỊCH CHỨNG KHOÁN,TỔ CHỨC,"""Sở GIAO DỊCH CHỨNG KHOÁN"" is a financial orga...","38dd0921ac731174a59ec6e94eeab28b,8574f3834cc03...",2,12,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,12,None,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


#### Read relationships

In [32]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 695


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,BÁN CHỨNG KHOÁN LÔ LẺ,MBS,1.0,Bán chứng khoán lô lẻ thông qua dịch vụ giao d...,[72c428d19c253035db6c6e314742ef8c],6deaefe707f84b3dbda979dea0d095ac,0,1,122,123
1,MBS,KHÁCH HÀNG,56.0,"The entity ""KHÁCH HÀNG"" is responsible for pro...","[008d65e4640a5208f1016c02bde466fa, 016f43c5ba9...",d053ea9432a24fb192e8d6aa993b0caa,1,122,118,240
2,MBS,CỔ PHIẾU,3.0,"""MBS"" is a company that does not repurchase ""C...","[1d3c82fc7b0630a6c46863b979a2af4c, 8574f3834cc...",a3e683d294ed42a28d60d09a36cbeb54,2,122,9,131
3,MBS,VSD,1.0,VSD hoàn thành chuyển quyền sở hữu chứng khoán...,[8574f3834cc035915cdfb9d2eedd0dc1],39887ca8567141d5b857b87a2bca4086,3,122,2,124
4,MBS,MBB,1.0,MBS không mua lại cổ phiếu lô lẻ MBB,[eab95c5e8ff4164dcfcbe12e60dd94ce],8df8563ab0394ee9a91b89dea7d59404,4,122,1,123


In [33]:
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

claims = read_indexer_covariates(covariate_df)

print(f"Claim records: {len(claims)}")
covariates = {"claims": claims}

Claim records: 443


#### Read community reports

In [34]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 92


/Users/dinhuyennhi/untitled folder/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/Users/dinhuyennhi/untitled folder/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,93,# Investors and Securities Trading\n\nThe comm...,3,7.5,Investors and Securities Trading,The impact severity rating is high due to the ...,The community consists of investors engaging i...,[{'explanation': 'Investors in this community ...,"{\n ""title"": ""Investors and Securities Trad...",eb7c99d4-aace-4c8f-ae72-82f109a80f97
1,94,# DANH MỤC and RỦI RO SẢN PHẨM in Stock Market...,3,7.5,DANH MỤC and RỦI RO SẢN PHẨM in Stock Market I...,The impact severity rating is high due to the ...,"The community revolves around DANH MỤC, a refe...",[{'explanation': 'DANH MỤC is a reference list...,"{\n ""title"": ""DANH M\u1ee4C and R\u1ee6I RO...",53ccec4a-9fe4-471b-b4ba-e72f98e0e1ab
2,83,# Investors and Securities Trading\n\nThe comm...,2,7.5,Investors and Securities Trading,The impact severity rating is high due to the ...,The community revolves around investors partic...,[{'explanation': 'NHÀ ĐẦU TƯ plays a central r...,"{\n ""title"": ""Investors and Securities Trad...",e1472932-91dc-457c-accd-5130127a42bc
3,84,# CW and Chứng Khoán Cơ Sở Community\n\nThe co...,2,7.5,CW and Chứng Khoán Cơ Sở Community,The impact severity rating is high due to the ...,"The community revolves around CW, a versatile ...",[{'explanation': 'CW plays a crucial role in t...,"{\n ""title"": ""CW and Ch\u1ee9ng Kho\u00e1n ...",2252012c-fbe0-4e58-bac3-b3fb77ba8c07
4,85,# CHỨNG QUYỀN and Financial Instruments\n\nThe...,2,7.5,CHỨNG QUYỀN and Financial Instruments,The impact severity rating is high due to the ...,"The community revolves around CHỨNG QUYỀN, a f...",[{'explanation': 'NHÀ ĐẦU TƯ are key participa...,"{\n ""title"": ""CH\u1ee8NG QUY\u1ec0N and Fin...",1e72838b-1638-4870-906d-ee84e27a3e9c


#### Read text units

In [35]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 240


,id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,72c428d19c253035db6c6e314742ef8c,Bán chứng khoán lô lẻ\nĐể thuận tiện cho việc ...,300,[d1d8f00369755e0157bedf841fd57ee7],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[6deaefe707f84b3dbda979dea0d095ac, c0866306dc8...","[1cc44f14-79d1-40d0-802c-550baab2fae3, f8144ad..."
1,8574f3834cc035915cdfb9d2eedd0dc1,� trực tiếp theo quy định của các Sở Giao dịch...,300,[d1d8f00369755e0157bedf841fd57ee7],"[4119fd06010c494caa07f439b333f4c5, 3671ea0dd4e...","[d053ea9432a24fb192e8d6aa993b0caa, a3e683d294e...",[959c8b7b-2770-4fd0-a80b-083373b31559]
2,eab95c5e8ff4164dcfcbe12e60dd94ce,ều).\nGiá giao dịch là giá sàn của cổ phiếu tạ...,300,[d1d8f00369755e0157bedf841fd57ee7],"[4119fd06010c494caa07f439b333f4c5, f7e11b0e297...","[a3e683d294ed42a28d60d09a36cbeb54, 8df8563ab03...",[8da5c012-47ec-417e-b4e9-01213aeb2e7d]
3,1d3c82fc7b0630a6c46863b979a2af4c,ịch từng giai đoạn.\nMBS không mua lại cổ phiế...,300,[d1d8f00369755e0157bedf841fd57ee7],"[4119fd06010c494caa07f439b333f4c5, e7ffaee9d31...","[a3e683d294ed42a28d60d09a36cbeb54, 12398f70065...","[007eae33-b6dc-4724-9e73-0a17719043a8, 419889f..."
4,9ea072a756077acc1c0b5f1c59da3857,ố rủi ro\n1. Giới thiệu hệ thống giao dịch trự...,300,[d1d8f00369755e0157bedf841fd57ee7],"[4119fd06010c494caa07f439b333f4c5, e7ffaee9d31...","[d053ea9432a24fb192e8d6aa993b0caa, 1b7a22f76f7...","[f329aab1-4da7-4e69-bf45-b9c6dbe2de9d, a0b2371..."


In [36]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = "gpt-3.5-turbo-0125"
embedding_model = "text-embedding-3-small"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

### Create local search context builder

In [37]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [38]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [39]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [129]:
question = "Thời gian giao dịch trái phiếu chính phủ tại Sở Giao dịch Chứng khoán Hà Nội như thế nào?" 
result = await search_engine.asearch(question)
print(result.response)

Thời gian giao dịch trái phiếu chính phủ tại Sở Giao dịch Chứng khoán Hà Nội diễn ra từ Thứ Hai đến Thứ Sáu hàng tuần, trừ ngày nghỉ lễ theo quy định của Bộ Luật Lao động. Phiên buổi sáng bắt đầu từ 9h00 và kết thúc vào 11h30, trong khi phiên buổi chiều diễn ra từ 13h00 đến 14h45. Điều này đảm bảo rằng nhà đầu tư có thể tham gia giao dịch trái phiếu chính phủ trong khung thời gian cụ thể hàng tuần tại Sở Giao dịch Chứng khoán Hà Nội. 

Thông tin này được trích dẫn từ nguồn [Sources (200); Sources (228); Sources (201)].


#### Inspecting the context data used to generate the response

In [130]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,501,SỞ GIAO DỊCH CHỨNG KHOÁN HÀ NỘI,Sở Giao dịch Chứng khoán Hà Nội là nơi tổ chức...,1,True
1,500,TRÁI PHIẾU CHÍNH PHỦ,Trái phiếu chính phủ là một trong các sản phẩm...,2,True
2,593,THỜI GIAN GIAO DỊCH,Thời gian giao dịch từ thứ Hai đến thứ Sáu hàn...,1,True
3,551,THỜI ĐIỂM THỰC HIỆN,Thời điểm thực hiện là thời điểm giao dịch đượ...,1,True
4,563,SỞ GIAO DỊCH,"Giao dịch trực tiếp tại Sở giao dịch, các Chi ...",2,True


In [131]:
result.context_data["relationships"].head()

,id,source,target,description,weight,rank,links,in_context
0,630,TRÁI PHIẾU CHÍNH PHỦ,SỞ GIAO DỊCH CHỨNG KHOÁN HÀ NỘI,Trái phiếu chính phủ được giao dịch tại Sở Gia...,1.0,3,1,True
1,9,MBS,SỞ GIAO DỊCH CHỨNG KHOÁN,Đường truyền kết nối từ MBS đến các Sở Giao dị...,1.0,134,3,True
2,13,MBS,GIAO DỊCH TRÁI PHIẾU,MBS tham gia vào việc bán cổ phiếu lẻ trong gi...,1.0,130,3,True
3,53,MBS,CHI NHÁNH/ PHÒNG GIAO DỊCH,Chi nhánh/ Phòng giao dịch của MBS là nơi khác...,1.0,123,3,True
4,302,KHÁCH HÀNG,GIAO DỊCH TRÁI PHIẾU,Khách hàng tham gia vào giao dịch trái phiếu v...,1.0,126,2,True


In [132]:
result.context_data["reports"]

,id,title,content
0,39,Trái Phiếu and Financial Market Entities,# Trái Phiếu and Financial Market Entities\n\n...
1,63,Types of Trading Transactions and Time Limits,# Types of Trading Transactions and Time Limit...


In [133]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

In [134]:
combined_text = " ".join(result.context_data["reports"]["content"])
total_tokens = len(tokenizer.encode(combined_text))

print(f"Output tokens: {total_tokens}")

Output tokens: 994


In [135]:
result.context_data["sources"]

,id,text
0,200,/2020\nTrái phiếu chính phủ\nI. QUY ĐỊNH GIAO ...
1,228,"/2022/NĐ-CP)\nĐVKD kiểm tra thông tin lệnh, th..."
2,211,ùng một đối tác giao dịch.\nBao gồm: Một giao ...
3,215,ưa thực hiện quy định trong Quy trình giao dịc...
4,109,đủ 6 ký tự)\nTài khoản theo số đẹp (mất phí):...
5,229,Phiên giao dịch chiều từ 13h00 - 15h00\nNghỉ t...
6,201,ngày nghỉ lễ theo quy định của Bộ Luật Lao độ...
7,208,tin khác có liên quan.\nThời gian kể từ ngày ...
8,216,9088.\nThời hạn thanh toán\nBù trừ đa phương: ...
9,14,iao dịch trái phiếu cho đối với giao dịch trải...


In [136]:
combined_text = " ".join(result.context_data["sources"]["text"])
total_tokens = len(tokenizer.encode(combined_text))
total_words = len(combined_text.split())

print(f"Doc words: {total_words}")
print(f"Doc tokens: {total_tokens}")

Doc words: 2529
Doc tokens: 5716


In [137]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

    id                    entity object_id status start_date end_date  \
0  266  SỞ GIAO DỊCH CHỨNG KHOÁN       MBS   TRUE       NONE     NONE   

                                         description  in_context  
0  MBS có thể tiếp nhận Dữ liệu cá nhân của Khách...        True  


### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [ ]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [ ]:
question_history = [
    "Mở tài khaonr kiểu gì?",
    "Cách mở tài khoản trực tiếp?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)

['- What are the steps to open a trading account directly at MBS?', '- How can customers open a trading account in person at MBS?', '- What is the process for opening a trading account directly at MBS?', '- Are there specific requirements for opening a trading account in person at MBS?', '- Can customers open a trading account face-to-face at MBS?']
